In [1]:
!pip install  -q "diffusers>=0.15.0" accelerate openvino-dev==2023.1.0.dev20230728 imageio[ffmpeg]

In [2]:
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained('damo-vilab/text-to-video-ms-1.7b')

2023-08-10 20:18:02.388802: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-10 20:18:02.390540: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 20:18:02.422602: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 20:18:02.424568: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 20:18:02.955148: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

## UNet

In [3]:
from openvino.runtime import Core
from conversion_utils import convert_unet

core = Core()
unet_converted_xml = convert_unet(pipe.unet)
unet_compiled = core.compile_model(unet_converted_xml)

## VAE

In [5]:
from conversion_utils import convert_vae_modules

vae_decoder_path, vae_post_quant_conv_path = convert_vae_modules(pipe.vae.decoder, pipe.vae.post_quant_conv)
ov_vae_decoder = core.compile_model(vae_decoder_path)
ov_vae_post_quant_conv = core.compile_model(vae_post_quant_conv_path)

## Text encoder

In [6]:
from conversion_utils import convert_text_encoder

text_encoder_xml = convert_text_encoder(pipe.text_encoder)
ov_text_encoder = core.compile_model(text_encoder_xml)

## Build a pipeline

In [7]:
from pipeline import OVTextToVideoSDPipeline

ov_pipe = OVTextToVideoSDPipeline(ov_vae_decoder, ov_vae_post_quant_conv, ov_text_encoder, pipe.tokenizer, unet_compiled, pipe.scheduler)


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
import numpy as np
import imageio

frames = ov_pipe("Superman saving the world", num_inference_steps=25)['frames']

def to_video(frames: list[np.ndarray], fps: int) -> str:
    out_file = "output.mp4"
    writer = imageio.get_writer(out_file, format='FFMPEG', fps=fps)
    for frame in frames:
        writer.append_data(frame)
    writer.close()
    return out_file

filename = to_video(frames, 8)

  0%|          | 0/25 [00:00<?, ?it/s]

In [12]:
%%HTML
<video controls>
  <source src="output.mp4">
</video>

In [1]:
from IPython.display import Video

Video(filename="output.mp4", embed=True)